# Simplified soiling model demo and cleaning schedule assessment
The below code shows how to use the simplified soiling model for a given solar field and assess the impact of a desired cleaning schedule.

In [ ]:
import soiling_model.base_models as smb
import soiling_model.field_models as smf
import soiling_model.utilities as smu

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

d = "woomera_demo/"
file_params = d+"parameters.xlsx"
file_weather = d+'woomera_data.xlsx'

file_SF = d+'SF_woomera_SolarPILOT.csv'             # solar field of 48 sectors located in Woomera
climate_file = d+'woomera_location_modified.epw'    # only used for optical efficiency computation

n_trucks = 4         # number of trucks
n_cleans = 10        # number of cleanings in time interval
cleaning_rate = 330  # heliostats cleaned per truck per day used to control sectorisation of heliostat field, leave blank to calculate using parameter values

# Instantiate Model
Location and solar field parameters are imported and processed to set the required inputs to the subsequent sections of the model.

`dust_types` has to be chosen according to the dust measurements, it can be either "PMX", or "TSP"


In [ ]:
imodel = smf.simplified_field_model(file_params,file_SF, cleaning_rate=cleaning_rate)
sim_data = smb.simulation_inputs(file_weather,dust_type="PM10")
plant = smf.central_tower_plant()
plant.import_plant(file_params)
imodel.helios.sector_plot()

# Sun and heliostat angles
The model proceeds to compute the relevant sun angles (azimuth and zenith) that are required to then evaluate the movement of the heliostats depending on plant design and time of the day.

In [ ]:
imodel.sun_angles(sim_data)
imodel.helios_angles(plant)

# Calculate area lost during each time step due to soiling
For this simplified model, only the `calculate_delta_soiled_area`` is needed. Computation details described in [9] from the README.md file. No Mie weights are needed since the model uses no information about the size distribution on the mirror. Conversely, this means that different acceptance angles cannot be modelled. 

In [ ]:
imodel.calculate_delta_soiled_area(sim_data)

The deposition area flux can be computed and shown for a given pair of air temperature (°C) and wind speed (m/s).

In [ ]:
# airT = 20
# windS = 2.0
# experiment = 0
# heliostat_id = 0
# imodel.plot_area_flux(sim_data,experiment,heliostat_id,airT,windS,tilt=0.0)

# Specify cleaning schedule and simulate its impact
The cleaning schedule specified by $n_{trucks}$ and $n_{cleans}$ is subsequently applied. This follows the *Heuristic Approach* described in [3] from README.md. Eventually, the reflectance losses are computed.

In [ ]:
cleans = smu.simple_annual_cleaning_schedule(imodel.helios.tilt[0].shape[0],n_trucks,n_cleans,dt=sim_data.dt[0]/3600.00)
cleans = imodel.reflectance_loss(sim_data,{0:cleans})

# Analysis and visulization of the results

The optical efficiency is computed for every sector (as the average of each heliostat belonging to the sector) in the field using a look-up table with `n_az` solar azimuth angles and `n_el` solar elevation angles. It eventually computes the average for the whole solar field.

In [ ]:
soiling_factor = imodel.helios.soiling_factor[0] # zero for the first "run"
field_average_soiling_factor = np.mean(soiling_factor,axis=0)
imodel.optical_efficiency(plant,sim_data,climate_file,verbose=True,n_az=10,n_el=10)
field_average_clean_optical_efficiency = np.mean(imodel.helios.optical_efficiency[0]*imodel.helios.nominal_reflectance,axis=0)

The below computes the resulting optical efficiency for each sector in soiled conditions. An average value for the whole field is also computed.

In [ ]:
soiled_optical_efficiency = imodel.helios.optical_efficiency[0]*soiling_factor*imodel.helios.nominal_reflectance
field_average_soiled_optical_efficiency = np.mean(soiled_optical_efficiency,axis=0)

The next section allows to plot the results of the model, including the average soiling factor, the soiling factor for a selected sector, and the optical efficiency of the solar field both in clean and soiled state.

In [ ]:
t = sim_data.time[0]
sec_plot = 0 # choose sector for plot

t0 = 96
t1 = 96+96 # hours of the year for the zoomed-in plot

# examine field soiling - field average
fig, ax = plt.subplots()
ax.plot_date(t,field_average_soiling_factor,"-")
ax.set_xlim(t[t0],t[t1])
ax.set_xlabel("Time")
ax.set_ylabel("Field-Averaged Soiling Factor [-]")
plt.xticks(rotation=45)

# examine sector soiling 
fig, ax = plt.subplots()
ax.plot_date(t,soiling_factor[sec_plot],"-")
ax.set_xlim(t.iloc[0],t.iloc[-1])
ax.set_xlabel("Time")
ax.set_ylabel("Soiling Factor [-]")
ax.set_title("Sector {0:d}".format(sec_plot))
plt.xticks(rotation=45)

In [ ]:
# examine optical efficiency - field average
t0 = 96
t1 = 96+96 # hours of the year for the zoomed-in plot

# t0 = 4000
# t1 = 4000+96 # hours of the year for the zoomed-in plot

fig, ax = plt.subplots(ncols=2,sharey=True,figsize=(12,5))
ax[0].plot_date(t,field_average_clean_optical_efficiency,"-",label='clean')
ax[0].plot_date(t,field_average_soiled_optical_efficiency,"-",label='soiled')
ax[0].set_xlim(t.iloc[0],t.iloc[-1])
ax[0].set_xlabel("Time")
ax[0].set_ylabel("Field-Averaged Optical Efficiency [-]")
ax[0].set_title("Full year")
for label in ax[0].get_xticklabels():
    label.set(rotation=45)

ax[1].plot_date(t,field_average_clean_optical_efficiency,"-",label='clean')
ax[1].plot_date(t,field_average_soiled_optical_efficiency,"-",label='soiled')
ax[1].set_xlim(t[t0],t[t1+10])
ax[1].set_xlabel("Time")
ax[1].set_title("Zoom in")
plt.xticks(rotation=45)
plt.legend()

The next plot allows to assess the tilt angle of heliostats, chosing the time frame in hours between `h_start` and `h_end`

In [ ]:
# examine tilts
h_start = 0
h_end = 48

Nhelios = imodel.helios.x.shape[0]
tilt = imodel.helios.tilt[0]
fig, ax = plt.subplots()
for h in range(Nhelios):
    ax.plot_date(t,tilt[h,:],"-")

ax.set_xlim(t[h_start],t[h_end])
ax.set_xlabel("Time")
ax.set_ylabel("Tilt angle (deg)")
plt.xticks(rotation=45)